<a href="https://colab.research.google.com/github/eduardosilva218/waifu-recommendation-system/blob/main/waifu_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd

waifus=pd.read_csv('pokewaifuset.csv')
waifus=waifus.pivot_table(columns="user", index="waifu_pokemon", values="rating")
waifus.fillna(0, inplace=True)
waifus

user,usuario_1,usuario_2,usuario_3
waifu_pokemon,,,
bulbassaur_waifu,6,10,3
eevee_waifu,4,10,3
flareon_waifu,4,10,1
gardevoir_waifu,8,3,8
pikachu_waifu,5,3,9
psyduck_waifu,3,2,8
roserade_waifu,10,4,8
slowpoke_waifu,4,9,2
squirtle_waifu,4,5,7


### with library

In [30]:
from sklearn.neighbors import NearestNeighbors

model_of_library = NearestNeighbors()
model_of_library.fit(waifus.values)

def predict_from_model_of_library(waifu_choosed, number_of_predicts = 3):
  def search_index_of_waifu(waifu_name_choosed):
    for index, waifu_name in enumerate(waifus.index):
      if waifu_name == waifu_name_choosed:
        return index
    raise Exception('No waifu detected.')

  index_waifu_choosed = search_index_of_waifu(waifu_choosed)
  waifu_value = waifus.iloc[index_waifu_choosed,:].values.reshape(1, -1)
  suggestions = model_of_library.kneighbors(waifu_value, number_of_predicts+1, return_distance=False)
  suggestions_to_return = []
  for suggestion in suggestions[0]:
    if index_waifu_choosed == suggestion:
      continue
    suggestions_to_return.append(waifus.index[suggestion])
  
  return suggestions_to_return

In [24]:
predict_from_model_of_library('gardevoir_waifu')

['roserade_waifu', 'pikachu_waifu', 'victribel_waifu']

### no library

In [43]:
import math

def get_euclidian_distance(vector_one, vector_two):
  result = 0
  for index in range(len(vector_one)):
    result += math.pow(vector_one[index]-vector_two[index], 2)
  return math.sqrt(result)

def get_manhattan_distance(vector_one , vector_two):
  result = 0
  for index in range(len(vector_one)):
    result += abs(vector_one[index]-vector_two[index])
  return result

def get_supremum_distance(vector_one, vector_two):
  values = []
  for index in range(len(vector_one)):
    values.append(abs(vector_one[index]-vector_two[index]))
  return max(values)

def get_minkowski_distance(vector_one, vector_two, p : float = 2):
  metrics = {'1': get_manhattan_distance,
               '2': get_euclidian_distance, 
               'inf': get_supremum_distance}
  for key in metrics:
    p = str(p)
    if key == p:
      return metrics[p](vector_one, vector_two)
  return None

class MyKNearestNeighbors():
  def fit(self, X):
    self.X = X
 
  def get_knearest_neighbors(self, X, number_of_predicts : int = 3):
    distances_and_values = []
    for index, vector in enumerate(self.X):
      for value in X:
        distance = get_minkowski_distance(vector, value)
        distances_and_values.append({'distance' : distance, 
                          'value' : vector, 'index' : index})
    result = sorted(distances_and_values, key=lambda d: d['distance'])
    return result[:number_of_predicts]

my_model = MyKNearestNeighbors()
my_model.fit(waifus.values)

def predict_from_my_library(waifu_choosed, number_of_predicts = 3):
  def search_index_of_waifu(waifu_name_choosed):
    for index, waifu_name in enumerate(waifus.index):
      if waifu_name == waifu_name_choosed:
        return index
    raise Exception('No waifu detected.')

  index_waifu_choosed = search_index_of_waifu(waifu_choosed)
  waifu_value = waifus.iloc[index_waifu_choosed,:].values.reshape(1, -1)
  suggestions = my_model.get_knearest_neighbors(waifu_value, number_of_predicts+1)
  suggestions_to_return = []
  for suggestion in suggestions:
    if index_waifu_choosed == suggestion['index']:
      continue
    waifu_to_return = waifus.index[suggestion['index']]
    suggestions_to_return.append(waifu_to_return)
  
  return suggestions_to_return

In [28]:
  predict_from_my_library('gardevoir_waifu')

['roserade_waifu', 'pikachu_waifu', 'victribel_waifu']

### Comparasion

In [42]:
print('meu resultado:\n')
for suggestion in my_model.get_knearest_neighbors([[4,10,3]], 3):
  print(suggestion)

print('\nresultado da lib:\n')
print(model_of_library.kneighbors([[4,10,3]], 3))

meu resultado:

{'distance': 0.0, 'value': array([ 4, 10,  3]), 'index': 1}
{'distance': 1.4142135623730951, 'value': array([4, 9, 2]), 'index': 7}
{'distance': 2.0, 'value': array([ 6, 10,  3]), 'index': 0}

resultado da lib:

(array([[0.        , 1.41421356, 2.        ]]), array([[1, 7, 2]]))
